In [1]:
from fit import *
import importlib

In [2]:
lower = 0.1
upper = 20
def ShiftFunction(params):
    minScale = lower
    commonParams = [param + np.sign(param) * minScale for param in params]
    return commonParams

def ShiftFunctionModular(params):
    minScale = lower
    commonParams = params[:-2]
    tParams = params[-2:]
    commonParams = [param + np.sign(param) * minScale for param in commonParams]
    return np.concatenate((commonParams, tParams))

In [3]:
# Notice numberOfParams does not need to count modular tau.
# modelName = "model.TpGroup.modelTp(3,2)"
modelName = "model.TpGroup.modelTp(1,2)"
# modelName = "model.TpGroup.model2T(3,2)"
modelModule = importlib.import_module(modelName)
observables=CKMSystem(modelModule.YuMatrix,modelModule.YdMatrix,modelModule.numberOfParams,dCPResult=True)
costFunction = CostFunction(observables, quarkCPExpValList ,quarkCPDivValList, shiftFunction=ShiftFunctionModular, modelType="modular", upper=upper)

In [4]:
scanN = 50
migradN = 100000
itrN = 50
fitResults = []
for i in range(scanN):
    fit = Minuit(costFunction, costFunction.InitParams())
    fit.limits = costFunction.parameterBounds
    fit.strategy = 2
    try:
        fit.migrad(migradN)
        fitResults.append(np.asarray(fit.values))
    except:
        continue

fitResult = min(fitResults, key=lambda x: costFunction(x))

# for the best fit, fit the point around it
convergingCount = 0
for i in range(itrN):
    OldResult = fitResult
    fit = Minuit(costFunction, fitResult)
    fit.limits = costFunction.parameterBounds
    fit.strategy = 2
    try:
        fit.migrad(migradN)
    except:
        continue
    fitResultNew = np.asarray(fit.values)
    fitResult = min([fitResultNew, fitResult], key=lambda x: costFunction(x))
    if np.all(OldResult == fitResultNew):
        convergingCount += 1
    if convergingCount > 3:
        break

print("==========================================================")
print("Number of parameters:")
print(costFunction.numberOfParams)
print("==========================================================")
costFunction.calResult.Print(ShiftFunction(fitResult))
print("===============----------------------------===============")
costFunction.Print(fitResult)
print("===============----------------------------===============")
print("Number of parameters:")
print(costFunction.numberOfParams)

Number of parameters:
2
Observabls Result:
s12 :  0.3973965711278097
s23 :  0.09584859687437762
s13 :  0.00560995029149322
mURmC :  1.5157577330124386e-16
mCRmT :  0.06198685548232541
mDRmS :  0.6461201073606008
mSRmB :  0.6074890863239658
QdCP :  1.2368904812682324
===============----------------------------===============
Fit Result:
-0.100000 -1.732023 -0.499998 0.866029
----------------------------------------------------------
Sigma Away:
s12 :  0.00021265293413670643
s23 :  1599.9998036290908
s13 :  176.80220378984814
mURmC :  10.346944444435463
mCRmT :  219876.6496795353
mDRmS :  23452.10299934692
mSRmB :  232137.76199582408
QdCP :  0.004513423424241738
----------------------------------------------------------
total chi-sqr:  477253.668352646
===============----------------------------===============
Number of parameters:
2
